In [1]:
#0.30
import pandas as pd
import pickle
import numpy as np 
import warnings                    
warnings.filterwarnings("ignore")

In [2]:
fint=pd.read_pickle("FinTech2_balanced.pkl")

In [3]:
fint.shape

(10612, 250)

In [4]:
# stratified sampling
#sam = fint.groupby('Target', group_keys=False).apply(lambda x: x.sample(frac=0.30))

In [5]:
df_dates = fint.filter(regex='DATE', axis=1)

# Define the fixed date
fixed_date = pd.to_datetime('2000-01-01')

# Iterate through the columns in df_dates
for column in df_dates.columns:
    # Convert the column to Timestamp data type
    df_dates[column] = pd.to_datetime(df_dates[column])
    
    # Calculate the difference in months from the fixed date and store it as a new column
    df_dates[f'months_since_2000_{column}'] = (df_dates[column].dt.year - fixed_date.year) * 12 + df_dates[column].dt.month - fixed_date.month
 

In [6]:
df_dates[:10]

,MEAS_ACAG_DATE_CONTACT_LAST,MEAS_ACCL_LAST_CONTACT_STEP_DATE,MEAS_ACCL_LAST_CONTACT_STR_DATE,MEAS_ACCL_LAST_KEPT_BROKEN_DATE,MEAS_ACCH_STATUS_DELINQ_DATE,MEAS_ACCL_PREVIOUS_RETURN_DATE,MEAS_ACCL_DATE_CAPITALISED,MEAS_ACCL_DATE_WRITEOFF_TOBE,MEAS_ACCH_DATE_LITIGATION_LATEST,MEAS_ACCL_SPECIAL_TREAT_DATE,...,months_since_2000_MEAS_ACCL_PHONE_RPC_DATE_FIRST,months_since_2000_MEAS_ACCL_PHONE_RPC_DATE_LATEST,months_since_2000_MEAS_CUSL_MIN_COMMUNICATION_DATE_3M,months_since_2000_MEAS_CUSL_ADMINISTRATION_STATUS_DATE,months_since_2000_MEAS_CUSL_PRIORITY_CUSTOMER_STATUS_DATE,months_since_2000_MEAS_CUSL_LAST_PAYMENT_DATE,months_since_2000_MEAS_CUSL_LAST_DEFAULT_DATE,months_since_2000_MEAS_CUSL_FIRST_DEFAULT_DATE,months_since_2000_MEAS_CUSH_BIRTHDATE,months_since_2000_MEAS_CUSH_DATE_NOMINAL
0,2019-11-22,2019-04-17,2019-04-17,2008-09-18,2018-07-01,2019-11-08,1900-01-01,1900-01-01,2013-01-22,2018-07-01,...,NaN,NaN,236,222,225.0,114.0,162.0,116.0,-368,238
1,2019-05-30,2019-04-09,2019-04-09,NaT,2018-07-01,2019-04-08,1900-01-01,1900-01-01,1900-01-01,2018-10-02,...,NaN,NaN,231,222,225.0,128.0,143.0,126.0,-406,232
2,2018-09-04,2019-04-09,2019-04-09,2007-11-16,2019-12-04,2019-08-30,1900-01-01,1900-01-01,1900-01-01,2018-10-02,...,NaN,NaN,237,239,NaN,144.0,145.0,145.0,-550,239
3,2018-11-10,2019-02-11,2019-02-11,2011-08-10,2018-11-10,2018-11-14,1900-01-01,1900-01-01,2013-07-04,2018-07-01,...,225.0,225.0,231,226,225.0,135.0,166.0,137.0,-177,233
4,2019-06-27,2019-04-17,2019-04-17,2014-07-14,2018-07-01,2019-02-05,1900-01-01,1900-01-01,1900-01-01,2018-07-01,...,NaN,NaN,231,222,NaN,128.0,127.0,127.0,-231,233
5,2019-10-29,2019-04-17,2019-04-17,2010-09-22,2018-07-01,2018-12-20,1900-01-01,1900-01-01,1900-01-01,2018-07-01,...,NaN,NaN,235,222,NaN,125.0,121.0,119.0,-207,237
6,2019-02-20,2019-02-23,2019-02-20,2008-02-14,2018-07-01,2018-07-02,1900-01-01,1900-01-01,1900-01-01,2018-07-01,...,NaN,NaN,227,222,NaN,141.0,116.0,109.0,-247,229
7,2019-01-29,2019-02-01,2019-01-29,2008-07-18,2018-07-01,2018-07-02,1900-01-01,1900-01-01,1900-01-01,2018-07-01,...,NaN,NaN,226,222,NaN,108.0,110.0,109.0,-329,228
8,2019-03-18,2019-04-09,2019-04-09,2010-01-08,2018-07-01,2019-04-08,1900-01-01,1900-01-01,2018-05-11,2018-07-01,...,NaN,NaN,230,222,225.0,120.0,121.0,119.0,-636,232
9,2019-09-25,2019-04-17,2019-04-17,NaT,2018-12-19,2018-11-14,1900-01-01,1900-01-01,2009-06-09,2018-07-01,...,227.0,236.0,235,227,225.0,108.0,103.0,102.0,-223,236


In [7]:
# Select and keep only the columns that start with 'months_since_2018' from df_dates
df_dates = df_dates.filter(like='months_since_2000', axis=1)

# Drop the original date columns from the sam DataFrame
columns_to_drop = df_dates.columns.str.replace('months_since_2000_', '')
fint.drop(columns=columns_to_drop, inplace=True)

# Concatenate the df_dates DataFrame with the modified sam DataFrame
fint = pd.concat([fint, df_dates], axis=1)

In [9]:
# replace the special characters of categorical columns
categ = fint.select_dtypes(include=['object']).columns
for col in categ:
    fint[col] = fint[col].str.replace('[%/_\-,.:&€=+\'">]', ' ', regex=True)

In [10]:
numer = fint.select_dtypes(include=[np.number])

In [12]:
for col in numer.columns:
    fint[col].fillna(fint[col].median(), inplace=True)

In [13]:
# calculate percentage of each class of target variable
class_counts = fint['Target'].value_counts(normalize=True) * 100
print(class_counts)

Target
0    50.0
1    50.0
Name: proportion, dtype: float64


In [14]:
total = fint.isnull().sum().sort_values(ascending=False)
percent = (fint.isnull().sum()/fint.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
MEAS_ACCL_NBR_OF_KEPT_PROMISES,3088,0.290991
IDacc,0,0.000000
MEAS_CUSL_PROFESSION_GROUPING,0,0.000000
MEAS_CUSL_COUNT_RPC,0,0.000000
MEAS_CUSL_COUNT_NO_CONTACT,0,0.000000
MEAS_CUSL_REFUSE,0,0.000000
MEAS_CUSL_PROMISE,0,0.000000
MEAS_CUSL_COMPLAIN,0,0.000000
MEAS_CUSL_COMMUNICATION_ATTEMPT,0,0.000000
MEAS_CUSL_RESPONDED,0,0.000000


In [15]:
mean_many_nan = ['MEAS_ACCL_NBR_OF_KEPT_PROMISES'] 
for col in mean_many_nan:
    fint[col].fillna(0, inplace=True)

In [16]:
total = fint.isnull().sum().sort_values(ascending=False)
percent = (fint.isnull().sum()/fint.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

,Total,Percent
IDacc,0,0.0
MEAS_CUSL_MAX_PERSON_CONTACT_TYPE,0,0.0
MEAS_CUSL_PHONE_RPC_COUNT,0,0.0
MEAS_CUSL_COUNT_RPC,0,0.0
MEAS_CUSL_COUNT_NO_CONTACT,0,0.0


In [18]:
fint.to_pickle("FinTech3_2000_3_balanced.pkl")  